## Deliverable 2. Create a Customer Travel Destinations Map.

In [48]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [18]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("c:/users/Andrew/world_weather_analysis/Weather_Database/WeatherPy_Database.csv")
city_data_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Atuona,PF,-9.8000,-139.0333,79.74,73,26,11.45,scattered clouds
1,1,Provideniya,RU,64.3833,-173.3000,33.66,93,100,19.30,snow
2,2,Norman Wells,CA,65.2820,-126.8329,1.40,66,75,20.71,broken clouds
3,3,Khandbari,NP,27.3747,87.2039,56.25,42,2,3.62,clear sky
4,4,Puerto Ayora,EC,-0.7393,-90.3518,84.20,74,20,14.97,light rain
5,5,General Pico,AR,-35.6566,-63.7568,88.50,32,91,10.65,moderate rain
6,6,Arya,PK,33.6000,73.0667,63.00,66,2,5.50,clear sky
7,7,Vestmannaeyjar,IS,63.4427,-20.2734,43.97,81,88,25.72,overcast clouds
8,8,Rikitea,PF,-23.1203,-134.9692,80.47,81,100,19.91,moderate rain
9,9,Thompson,CA,55.7435,-97.8558,42.80,49,20,12.66,few clouds


In [12]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature high you would like for your trip? "))
max_temp = float(input("What is the maximum temperature high you would like for your trip? "))

What is the minimum temperature high you would like for your trip? 75
What is the maximum temperature high you would like for your trip? 80


In [19]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"]>= min_temp)]
preferred_cities_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Atuona,PF,-9.8000,-139.0333,79.74,73,26,11.45,scattered clouds
10,10,Bien Hoa,VN,10.9500,106.8167,78.80,69,40,4.61,scattered clouds
12,12,Basco,PH,20.4487,121.9702,75.74,84,51,19.89,broken clouds
20,20,Salalah,OM,17.0151,54.0924,77.00,50,0,2.30,clear sky
22,22,Monrovia,LR,6.3005,-10.7969,77.00,94,20,8.05,few clouds
...,...,...,...,...,...,...,...,...,...,...
656,656,Isla Mujeres,MX,21.2311,-86.7310,78.80,57,75,12.66,broken clouds
663,663,Colomi,BO,-17.3500,-65.8667,77.00,27,20,10.36,few clouds
670,670,Nagapattinam,IN,10.7667,79.8333,76.69,90,6,6.80,clear sky
678,678,Pundaguitan,PH,6.3711,126.1689,79.83,77,100,8.23,overcast clouds


In [16]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                120
City                   120
Country                120
Lat                    120
Lng                    120
Max Temp               120
Humidity               120
Cloudiness             120
Wind Speed             120
Current Description    120
dtype: int64

In [21]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
pref_cities_df = preferred_cities_df.dropna()
pref_cities_df.count()

City_ID                120
City                   120
Country                120
Lat                    120
Lng                    120
Max Temp               120
Humidity               120
Cloudiness             120
Wind Speed             120
Current Description    120
dtype: int64

In [38]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = pref_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Atuona,PF,79.74,scattered clouds,-9.8000,-139.0333,
10,Bien Hoa,VN,78.80,scattered clouds,10.9500,106.8167,
12,Basco,PH,75.74,broken clouds,20.4487,121.9702,
20,Salalah,OM,77.00,clear sky,17.0151,54.0924,
22,Monrovia,LR,77.00,few clouds,6.3005,-10.7969,


In [43]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found")
print("Done searching for hotels")     

Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Done searching for hotels


In [72]:
# 7. Find rows where hotel was not found found and delete row from data frame.
hotel_df["Hotel Name"] = hotel_df["Hotel Name"].replace("",np.nan)
clean_hotel_df = hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Atuona,PF,79.74,scattered clouds,-9.8000,-139.0333,Villa Enata
10,Bien Hoa,VN,78.80,scattered clouds,10.9500,106.8167,OYO 991 Duy Anh Hotel
12,Basco,PH,75.74,broken clouds,20.4487,121.9702,Nathaniel's Lodge
20,Salalah,OM,77.00,clear sky,17.0151,54.0924,HAMDAN PLAZA HOTEL SALALAH
22,Monrovia,LR,77.00,few clouds,6.3005,-10.7969,Sunset Inn
...,...,...,...,...,...,...,...
656,Isla Mujeres,MX,78.80,broken clouds,21.2311,-86.7310,Hotel La Joya Isla Mujeres
663,Colomi,BO,77.00,few clouds,-17.3500,-65.8667,Residencial EL TURISMO
670,Nagapattinam,IN,76.69,clear sky,10.7667,79.8333,Sathya Lodge
678,Pundaguitan,PH,79.83,overcast clouds,6.3711,126.1689,Leonardo Family Beach House 1


In [73]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [74]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>

<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>

</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [81]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(45.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))